In [1]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
import re
from lxml import etree
import pandas as pd
import time
import threading #线程
import queue  #线程
from multiprocessing import cpu_count
from multiprocessing import Process,Queue #进程
from multiprocessing import Pool,Manager  #进程
import random
import http.cookiejar as cookielib

In [8]:
#获取页面
link="http://www.santostang.com/"
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
print(r.text)
#提取需要的数据
soup=BeautifulSoup(r.text,'lxml')
title=soup.find("h1",class_='post-title').a.text.strip()
print(title)
#存储数据
with open('title.txt',"a+") as f:
    f.write(title)
    f.close()

<!DOCTYPE html>
<html lang="zh-CN">
<head>
<meta charset="UTF-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
<title>Santos Tang</title>
<meta name="description" content="Python网络爬虫：从入门到实践 官方网站及个人博客" />
<meta name="keywords" content="Python网络爬虫, Python爬虫, Python, 爬虫, 数据科学, 数据挖掘, 数据分析, santostang, Santos Tang, 唐松, Song Tang" />
<link rel="apple-touch-icon" href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_32.png">
<link rel="apple-touch-icon" sizes="152x152" href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_152.png">
<link rel="apple-touch-icon" sizes="167x167" href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_167.png">
<link rel="apple-touch-icon" sizes="180x180" href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_180.png">
<link rel="icon" href="http://www.santostang.com/wp-content/themes/S

### 静态网页抓取（HTML）

In [7]:
r=requests.get("http://santostang.com/")
r.encoding #文本编码
r.status_code #响应状态码
r.text #字符串内容
r.content #字节内容
r.json #内置JSON解码器

<bound method Response.json of <Response [200]>>

In [21]:
#传递URL参数
key_dict={'key1':'value1','key2':'value2'}
url=requests.get('http://httpbin.org/get',params=key_dict)
url.url #传递后的url
print(url.text) #字符串方式的响应体

{
  "args": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.21.0"
  }, 
  "origin": "183.234.253.75, 183.234.253.75", 
  "url": "https://httpbin.org/get?key1=value1&key2=value2"
}



### 定制请求头

In [22]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "Host":"www.santostang.com"}
key_dict={'key1':'value1','key2':'value2'}
url=requests.get('http://httpbin.org/get',params=key_dict,headers=headers)
url.url #传递后的url
print(url.text) #字符串方式的响应体

{
  "args": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "www.santostang.com", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
  }, 
  "origin": "183.234.253.75, 183.234.253.75", 
  "url": "https://www.santostang.com/get?key1=value1&key2=value2"
}



In [24]:
key_dict={'key1':'value1','key2':'value2'}
r=requests.post('http://httpbin.org/post',data=key_dict,headers=headers)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "23", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "www.santostang.com", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
  }, 
  "json": null, 
  "origin": "183.234.253.75, 183.234.253.75", 
  "url": "https://www.santostang.com/post"
}



### TOP250豆瓣电影名称

In [26]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "Host":"movie.douban.com"}
movies=[]
for i in range(0,10):
    url="https://movie.douban.com/top250?start={0}".format(i*25)
    r=requests.get(url,headers=headers,timeout=10)
    print(str(i+1)+"页响应状态码:",r.status_code)
    soup=BeautifulSoup(r.text,"lxml")
    div_list=soup.find_all('div',class_='hd')
    for d in div_list:
        movie=d.a.span.text.strip()
        movies.append(movie)

1页响应状态码: 200
2页响应状态码: 200
3页响应状态码: 200
4页响应状态码: 200
5页响应状态码: 200
6页响应状态码: 200
7页响应状态码: 200
8页响应状态码: 200
9页响应状态码: 200
10页响应状态码: 200


In [29]:
movies

['肖申克的救赎',
 '霸王别姬',
 '这个杀手不太冷',
 '阿甘正传',
 '美丽人生',
 '泰坦尼克号',
 '千与千寻',
 '辛德勒的名单',
 '盗梦空间',
 '忠犬八公的故事',
 '机器人总动员',
 '三傻大闹宝莱坞',
 '海上钢琴师',
 '放牛班的春天',
 '楚门的世界',
 '大话西游之大圣娶亲',
 '星际穿越',
 '龙猫',
 '教父',
 '熔炉',
 '无间道',
 '疯狂动物城',
 '当幸福来敲门',
 '怦然心动',
 '触不可及',
 '蝙蝠侠：黑暗骑士',
 '乱世佳人',
 '活着',
 '控方证人',
 '少年派的奇幻漂流',
 '指环王3：王者无敌',
 '天堂电影院',
 '鬼子来了',
 '天空之城',
 '摔跤吧！爸爸',
 '十二怒汉',
 '飞屋环游记',
 '大话西游之月光宝盒',
 '哈尔的移动城堡',
 '搏击俱乐部',
 '罗马假日',
 '闻香识女人',
 '末代皇帝',
 '辩护人',
 '寻梦环游记',
 '窃听风暴',
 '素媛',
 '两杆大烟枪',
 '死亡诗社',
 '飞越疯人院',
 '指环王2：双塔奇兵',
 '教父2',
 'V字仇杀队',
 '指环王1：魔戒再现',
 '狮子王',
 '美丽心灵',
 '饮食男女',
 '海豚湾',
 '情书',
 '钢琴家',
 '本杰明·巴顿奇事',
 '看不见的客人',
 '黑客帝国',
 '美国往事',
 '哈利·波特与魔法石',
 '大闹天宫',
 '西西里的美丽传说',
 '小鞋子',
 '拯救大兵瑞恩',
 '让子弹飞',
 '致命魔术',
 '七宗罪',
 '音乐之声',
 '被嫌弃的松子的一生',
 '低俗小说',
 '猫鼠游戏',
 '沉默的羔羊',
 '天使爱美丽',
 '蝴蝶效应',
 '勇敢的心',
 '春光乍泄',
 '剪刀手爱德华',
 '心灵捕手',
 '穿条纹睡衣的男孩',
 '禁闭岛',
 '布达佩斯大饭店',
 '入殓师',
 '阿凡达',
 '幽灵公主',
 '致命ID',
 '加勒比海盗',
 '阳光灿烂的日子',
 '摩登时代',
 '断背山',
 '第六感',
 '重庆森林',
 '狩猎',
 '玛丽和马克思',
 '喜剧之王',
 '告白',
 '消失的爱人',
 '大鱼',
 '一一

### 动态网页抓取(AJAX)
#### 1,通过浏览器审查元素解析地址
#### 2，通过Selenium模拟浏览器抓取

In [16]:
link='https://api-zero.livere.com/v1/comments/list?callback=jQuery1124024106304402849466_1561794027986&limit=10&repSeq=4272904&requestPath=%2Fv1%2Fcomments%2Flist&consumerSeq=1020&livereSeq=28583&smartloginSeq=5154&_=1561794027988'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
# 从json数据中提取数据
json_data=r.json
# comments=json_data['data']['comments']
# for eachone in comments:
#     message=comments['eachone']['content']
#     print(message)


<bound method Response.json of <Response [200]>>


In [17]:
driver=webdriver.Chrome()
driver.get('https://www.dianping.com/search/category/7/10/p1')

### 解析网页
#### 1，正则表达式 :re.match  re.search  re.findall
#### 2，BeautifulSoup
#### 3，lxml

In [24]:
#正则表达式 re.match : 从字符串开头开始匹配
line="Fat cats are smarter than dogs,is it right?"
pattern=r'(.*)are(.*?)dogs'
m=re.match(pattern=pattern,string=line)
print("匹配整句话为：",m.group(0))
print("匹配第一个结果：",m.group(1))
print("匹配第二个结果：",m.group(2))
print("匹配的结果列表：：",m.groups())

匹配整句话为： Fat cats are smarter than dogs
匹配第一个结果： Fat cats 
匹配第二个结果：  smarter than 
匹配的结果列表：： ('Fat cats ', ' smarter than ')


In [25]:
#正则表达式 re.search  与match方法差不多
m_match=re.match("com","www.sjshfjs.com")
m_search=re.search("com","www.dsajfg.com")
print(m_match)
print(m_search)

None
<_sre.SRE_Match object; span=(11, 14), match='com'>


In [29]:
#正则表达式 re.findall
m_findall=re.findall('\d+',"1234the 4are bule 23535yellow34564")
m_findall

['1234', '4', '23535', '34564']

In [31]:
link='http://www.santostang.com'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
html=r.text
title_list=re.findall('<h1 class="post-title"><a href=.*?>(.*?)</a></h1>',html)
print(title_list)

['第四章 &#8211; 4.3 通过selenium 模拟浏览器抓取', '第四章 &#8211; 4.2 解析真实地址抓取', '第四章- 动态网页抓取 (解析真实地址 + selenium)', 'Hello world!']


In [34]:
#使用BeautifulSoup
link='http://www.santostang.com'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
soup=BeautifulSoup(r.text,"html.parser") #把代码转化为BeautifulSoup对象
title_list=soup.find_all('h1',class_='post-title')
for i in range(len(title_list)):
    title=title_list[i].a.text.strip()
    print('第{0}个文章的标题为：{1}'.format(i+1,title))

第1个文章的标题为：第四章 – 4.3 通过selenium 模拟浏览器抓取
第2个文章的标题为：第四章 – 4.2 解析真实地址抓取
第3个文章的标题为：第四章- 动态网页抓取 (解析真实地址 + selenium)
第4个文章的标题为：Hello world!


#### BeautifulSoup对象是一个复杂的树形结构
#### (1)遍历文档树 
#### (2) 搜索文档树
#### (3) CSS选择树

In [39]:
print(soup.prettify())#将代码进行美化

<!DOCTYPE html>
<html lang="zh-CN">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <title>
   Santos Tang
  </title>
  <meta content="Python网络爬虫：从入门到实践 官方网站及个人博客" name="description">
   <meta content="Python网络爬虫, Python爬虫, Python, 爬虫, 数据科学, 数据挖掘, 数据分析, santostang, Santos Tang, 唐松, Song Tang" name="keywords">
    <link href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_32.png" rel="apple-touch-icon"/>
    <link href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_152.png" rel="apple-touch-icon" sizes="152x152"/>
    <link href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_167.png" rel="apple-touch-icon" sizes="167x167"/>
    <link href="http://www.santostang.com/wp-content/themes/SongStyle-Two/images/icon_180.png" rel="apple-touch-icon" sizes="180x180"/>
    <link href="http://www.s

In [71]:
print(soup.header.h1)

<h1 id="name">Santos Tang</h1>


In [69]:
print(soup.header.div.contents,)

[<a href="http://www.santostang.com" title="Santos Tang"><img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/></a>]


In [70]:
for child in soup.header.div.children:
    print(child)

<a href="http://www.santostang.com" title="Santos Tang"><img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/></a>


In [72]:
for child in soup.header.div.descendants:
    print(child)

<a href="http://www.santostang.com" title="Santos Tang"><img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/></a>
<img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/>


In [73]:
a_tag=soup.header.div.a
a_tag.parent

<div class="avatar"><a href="http://www.santostang.com" title="Santos Tang"><img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/></a></div>

In [75]:
#搜索文档树
for tag in soup.find_all(re.compile('^h')):
    print(tag.name)

html
head
header
h1
h2
header
h1
header
h1
header
h1
header
h1
h4
h4
h4
h4
h4


In [80]:
#CSS选择器
soup.select("header h1")

[<h1 id="name">Santos Tang</h1>,
 <h1 class="post-title"><a href="http://www.santostang.com/2018/07/15/4-3-%e9%80%9a%e8%bf%87selenium-%e6%a8%a1%e6%8b%9f%e6%b5%8f%e8%a7%88%e5%99%a8%e6%8a%93%e5%8f%96/">第四章 – 4.3 通过selenium 模拟浏览器抓取</a></h1>,
 <h1 class="post-title"><a href="http://www.santostang.com/2018/07/14/4-2-%e8%a7%a3%e6%9e%90%e7%9c%9f%e5%ae%9e%e5%9c%b0%e5%9d%80%e6%8a%93%e5%8f%96/">第四章 – 4.2 解析真实地址抓取</a></h1>,
 <h1 class="post-title"><a href="http://www.santostang.com/2018/07/14/%e7%ac%ac%e5%9b%9b%e7%ab%a0%ef%bc%9a%e5%8a%a8%e6%80%81%e7%bd%91%e9%a1%b5%e6%8a%93%e5%8f%96-%e8%a7%a3%e6%9e%90%e7%9c%9f%e5%ae%9e%e5%9c%b0%e5%9d%80-selenium/">第四章- 动态网页抓取 (解析真实地址 + selenium)</a></h1>,
 <h1 class="post-title"><a href="http://www.santostang.com/2018/07/04/hello-world/">Hello world!</a></h1>]

In [84]:
print(soup.select("header > h1"))

[<h1 id="name">Santos Tang</h1>, <h1 class="post-title"><a href="http://www.santostang.com/2018/07/15/4-3-%e9%80%9a%e8%bf%87selenium-%e6%a8%a1%e6%8b%9f%e6%b5%8f%e8%a7%88%e5%99%a8%e6%8a%93%e5%8f%96/">第四章 – 4.3 通过selenium 模拟浏览器抓取</a></h1>, <h1 class="post-title"><a href="http://www.santostang.com/2018/07/14/4-2-%e8%a7%a3%e6%9e%90%e7%9c%9f%e5%ae%9e%e5%9c%b0%e5%9d%80%e6%8a%93%e5%8f%96/">第四章 – 4.2 解析真实地址抓取</a></h1>, <h1 class="post-title"><a href="http://www.santostang.com/2018/07/14/%e7%ac%ac%e5%9b%9b%e7%ab%a0%ef%bc%9a%e5%8a%a8%e6%80%81%e7%bd%91%e9%a1%b5%e6%8a%93%e5%8f%96-%e8%a7%a3%e6%9e%90%e7%9c%9f%e5%ae%9e%e5%9c%b0%e5%9d%80-selenium/">第四章- 动态网页抓取 (解析真实地址 + selenium)</a></h1>, <h1 class="post-title"><a href="http://www.santostang.com/2018/07/04/hello-world/">Hello world!</a></h1>]


In [85]:
print(soup.select("div > a"))

[<a href="http://www.santostang.com" title="Santos Tang"><img alt="Santos Tang" class="img-circle" src="http://www.santostang.com/wp-content/uploads/2019/06/me.jpg" width="50%"/></a>, <a data-placement="top" data-toggle="tooltip" href="https://weibo.com/santostang" rel="nofollow" target="_blank" title="Weibo"><i class="fab fa-weibo"></i></a>, <a data-placement="top" data-toggle="tooltip" href="https://www.linkedin.com/in/santostang" rel="nofollow" target="_blank" title="Linkedin"><i class="fab fa-linkedin"></i></a>, <a data-placement="top" data-toggle="tooltip" href="https://www.zhihu.com/people/santostang" rel="nofollow" target="_blank" title="Zhihu"><i class="fab fa-zhihu"></i></a>, <a data-placement="top" data-toggle="tooltip" href="https://github.com/santostang" rel="nofollow" target="_blank" title="GitHub"><i class="fab fa-github-alt"></i></a>, <a aria-label="ajax (2个项目);" class="tag-cloud-link tag-link-3 tag-link-position-1" href="http://www.santostang.com/tag/ajax/" style="color

In [86]:
soup.select('a[href^="http://www.santostang.com/"]')

[<a href="http://www.santostang.com/">首页</a>,
 <a href="http://www.santostang.com/aboutme/">关于我</a>,
 <a href="http://www.santostang.com/python%e7%bd%91%e7%bb%9c%e7%88%ac%e8%99%ab%e4%bb%a3%e7%a0%81/">爬虫书代码</a>,
 <a href="http://www.santostang.com/%e5%8a%a0%e6%88%91%e5%be%ae%e4%bf%a1/">加我微信</a>,
 <a href="http://www.santostang.com/2018/07/11/%e3%80%8a%e7%bd%91%e7%bb%9c%e7%88%ac%e8%99%ab%ef%bc%9a%e4%bb%8e%e5%85%a5%e9%97%a8%e5%88%b0%e5%ae%9e%e8%b7%b5%e3%80%8b%e4%b8%80%e4%b9%a6%e5%8b%98%e8%af%af/" target="_blank">《网络爬虫：从入门到实践》一书勘误</a>,
 <a href="http://www.santostang.com/2018/07/15/4-3-%e9%80%9a%e8%bf%87selenium-%e6%a8%a1%e6%8b%9f%e6%b5%8f%e8%a7%88%e5%99%a8%e6%8a%93%e5%8f%96/">第四章 – 4.3 通过selenium 模拟浏览器抓取</a>,
 <a href="http://www.santostang.com/category/python-%e7%bd%91%e7%bb%9c%e7%88%ac%e8%99%ab/" rel="category tag">Python 网络爬虫</a>,
 <a href="http://www.santostang.com/2018/07/15/4-3-%e9%80%9a%e8%bf%87selenium-%e6%a8%a1%e6%8b%9f%e6%b5%8f%e8%a7%88%e5%99%a8%e6%8a%93%e5%8f%96/#respond">没有评论<

### 使用lxml解析网页(XPath)

In [95]:
link='http://www.santostang.com/'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
html=etree.HTML(r.text)
title_list=html.xpath('//h1[@class="post-title"]/a/text()')
title_list

['第四章 – 4.3 通过selenium 模拟浏览器抓取',
 '第四章 – 4.2 解析真实地址抓取',
 '第四章- 动态网页抓取 (解析真实地址 + selenium)',
 'Hello world!']

In [32]:
name_list=[]
price_list=[]
price_area_list=[]
room_list=[]
area_list=[]
floor_list=[]
year_list=[]
broker_list=[]
address_list=[]
tags_list=[]

link='https://beijing.anjuke.com/sale/p'

for i in range(1,11):
    print("正在爬取第%d页。。。"%i)
    link_all=link+str(i)
    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
    r=requests.get(link_all,headers=headers)
    soup=BeautifulSoup(r.text,'lxml')
    house_list=soup.find_all("li",class_="list-item")

    for house in house_list:
        name=house.find("div",class_="house-title").a.text.strip()
        price=house.find("span",class_="price-det").text.strip()
        price_area=house.find("span",class_="unit-price").text.strip()
        room=house.find("div",class_="details-item").span.text
        area=house.find("div",class_="details-item").contents[3].text
        floor=house.find("div",class_="details-item").contents[5].text
        year=house.find("div",class_="details-item").contents[7].text
        broker=house.find("span",class_="broker-name broker-text").text.strip()
        address=house.find("span",class_="comm-address").text.strip()
        address=address.replace("  \n                    "," ")
        tag_list=house.find_all("span",class_="item-tags tag-others")
        tags=[i.text for i in tag_list]

        name_list.append(name)
        price_list.append(price)
        price_area_list.append(price_area)
        room_list.append(room)
        area_list.append(area)
        floor_list.append(floor)
        year_list.append(year)
        broker_list.append(broker)
        address_list.append(address)
        tags_list.append(tags)
    time.sleep(5)
#     print(name,price,price_area,room,area,floor,year,broker,address,tags)

正在爬取第1页。。。
正在爬取第2页。。。
正在爬取第3页。。。
正在爬取第4页。。。
正在爬取第5页。。。
正在爬取第6页。。。
正在爬取第7页。。。
正在爬取第8页。。。
正在爬取第9页。。。
正在爬取第10页。。。


In [33]:
columns=["房屋名称","价格","均价","几室几厅","面积","层数","建造年份","房屋中介","地址","标签"]
data=[name_list,price_list,price_area_list,room_list,area_list,floor_list,year_list,broker_list,address_list,tags_list]
df=pd.DataFrame(data=data)
df=df.T
df.columns=columns
df

,房屋名称,价格,均价,几室几厅,面积,层数,建造年份,房屋中介,地址,标签
0,首创上叠拼别墅，带50米露台，使用160米320万，精装修。,320万,36364元/m²,2室2厅,88m²,共3层,2013年建造,杨继东,首创澜茵山(别墅) 密云-密云城区-密关路,[]
1,精装修，首创上叠拼，带40平观景露台，三室，使用160平米。,298万,33864元/m²,3室2厅,88m²,共3层,2014年建造,孙海波,首创澜茵山(别墅) 密云-密云城区-密关路,[]
2,海怡庄园独立两居室！看房方便，交通便利。周边配套齐全欢迎咨询,175万,18454元/m²,2室1厅,94m²,共3层,2009年建造,丁保平,海怡庄园(西区) 密云-密云周边-京密路,[]
3,白河涧云秀花园月季园 精装修南北通透94平米 220万,220万,23404元/m²,2室2厅,94m²,共4层,2009年建造,谢兰凤,云秀花园公寓区 密云-密云城区-密溪路,"[素质住户, 安全性高, 品质小区]"
4,云秀公寓，低密度，环境好，精装二层，经典两居室，230万,230万,25275元/m²,2室2厅,91m²,共4层,2006年建造,傅君燕,云秀花园公寓区 密云-密云城区-密溪路,"[南北通透, 拎包入住, 低密度]"
5,云秀花园，精装修91米245万，好楼层，2室2厅1卫,245万,34507元/m²,2室2厅,71m²,共4层,2006年建造,景洋,云秀花园公寓区 密云-密云城区-密溪路,"[房型正, 拎包入住, 视野好]"
6,（云秀花园三居室）精装修，全明格局，满五年，随时看房,288万,25487元/m²,3室2厅,113m²,共4层,2006年建造,聂小爽,云秀花园公寓区 密云-密云城区-密溪路,"[拎包入住, 环境优美, 品质小区]"
7,城后小区 小三居 精装修 南北通透 业主急售 86平米190,190万,22093元/m²,2室1厅,86m²,共1层,2006年建造,王春杰,城后小区 密云-密云城区-城后东街60号,[]
8,下叠拼带花园，81平米三居室，410万，毛坯房随意装修,410万,50617元/m²,3室2厅,81m²,共4层,2014年建造,杨加齐,首创澜茵山(别墅) 密云-密云城区-密关路,"[环境优美, 安全性高, 品质小区]"
9,蓝河湾 独立两室 南北通透 阳光三层 邻近万象汇看房我有钥匙,220万,24719元/m²,2室2厅,89m²,共1层,2006年建造,陈军,蓝河湾 密云-密云周边-水源路,"[素质住户, 繁华地段]"


In [39]:
df.to_excel("./安居客房屋600间.xlsx",index=False)
df.to_csv("./安居客房屋600间2.csv",encoding="UTF-8") #乱码

### 爬虫实践：虎扑论坛

In [38]:
link="http://bbs.hupu.com/bxj-1"
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
soup=BeautifulSoup(r.content.decode("UTF-8"),"lxml")
post_list=soup.find_all("li",class_="")
post_list
# for post in post_list:
#     title=post.find("div",class_="titlelink")
# title

[<li><a href="https://www.hupu.com/">虎扑首页</a></li>,
 <li><a href="https://nba.hupu.com/">NBA</a></li>,
 <li><a href="https://cba.hupu.com/">CBA</a></li>,
 <li><a href="https://gg.hupu.com">电竞</a></li>,
 <li><a href="https://soccer.hupu.com">国际足球</a></li>,
 <li><a href="https://soccer.hupu.com/china/">中国足球</a></li>,
 <li><a href="http://racing.hupu.com/">赛车</a></li>,
 <li><a href="http://zb.hupu.com/" target="_blank">装备</a></li>,
 <li><a href="http://www.shihuo.cn/" target="_blank">识货</a></li>,
 <li><a href="https://bbs.hupu.com/">社区</a></li>,
 <li><a href="http://www.liangle.com" target="_blank">路人王</a></li>,
 <li><a href="https://bbs.hupu.com/all-gambia">首页</a></li>,
 <li>
 <a href="/all-gambia">步行街热帖</a>
 </li>,
 <li>
 <a class="on" href="/bxj">步行街主干道</a>
 </li>,
 <li>
 <a href="/selfie">晒晒照片</a>
 </li>,
 <li>
 <a href="/ent">影视</a>
 </li>,
 <li>
 <a href="/4856">娱乐圈</a>
 </li>,
 <li>
 <a href="/digital">数码</a>
 </li>,
 <li>
 <a href="/cars">车友交流</a>
 </li>,
 <li>
 <a href="/4846">啥破

### 多线程

In [41]:
class myThread(threading.Thread):
    def __init__(self,name,delay):
        threading.Thread.__init__(self)
        self.name=name
        self.delay=delay
    def run(self):
        print("Starting:"+self.name)
        print_time(self.name,self.delay)
        print("Exiting:"+self.name)
def print_time(threadName,delay):
    counter=0
    while counter < 3:
        time.sleep(delay)
        print(threadName,time.ctime())
        counter+=1
threads=[]
#创建新线程
thread1=myThread("Thread-1",1)
thread2=myThread("Thread-2",2)

#开启新线程
thread1.start()
thread2.start()

#添加线程到线程列表
threads.append(thread1)
threads.append(thread2)

#等待所有线程完成
for t in threads:
    t.join()
    
print("Exiting Main Thread")

Starting:Thread-1Starting:Thread-2

Thread-1 Sun Jun 30 23:35:08 2019
Thread-2 Sun Jun 30 23:35:09 2019
Thread-1 Sun Jun 30 23:35:09 2019
Thread-1 Sun Jun 30 23:35:10 2019
Exiting:Thread-1
Thread-2 Sun Jun 30 23:35:11 2019
Thread-2 Sun Jun 30 23:35:13 2019
Exiting:Thread-2
Exiting Main Thread


### 简单的多线程爬虫

In [2]:
link_list=[]
thread_list=[]
link_range_list=[(0,200),(201,400),(401,600),(601,800),(801,1000)]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
with open('alexa.txt','r') as f:
    f_list=f.readlines()
    for eachone in f_list:
        link=eachone.split('\t')[1]
        link=link.replace('\n','')
        link_list.append(link)
    f.close()

start=time.time()
class myThread(threading.Thread):
    def __init__(self,name,link_range):
        threading.Thread.__init__(self)
        self.name=name
        self.link_range=link_range
    def run(self):
        print("Starting:"+self.name)
        crawler(self.name,self.link_range)
        print("Exiting:"+self.name)
def crawler(threadName,link_range):
    for i in range(link_range[0],link_range[1]+1):
        try:
            r=requests.get(link_list[i],headers=headers,timeout=50)
            print(threadName,r.status_code,link_list[i])
        except Exception as e:
            print(threadName,'Error:',e)


#创建多线程
for i in range(1,6):
    thread=myThread('Thread-'+str(i),link_range_list[i-1])
    thread.start()
    thread_list.append(thread)
    
#等待所有线程完成
for thread in thread_list:
    thread.join()

end=time.time()
print("简单多线程爬虫的总时间为：",end-start)
print("Exiting Main Thread")

Starting:Thread-1Starting:Thread-2

Starting:Thread-3Starting:Thread-4
Starting:Thread-5

Thread-2 200 http://www.cnki.net
Thread-5 200 http://www.ef43.com.cn
Thread-1 200 http://www.baidu.com
Thread-3 200 http://www.yy.com
Thread-4 200 http://www.java.com
Thread-3 200 http://www.duowan.com
Thread-4 200 http://www.hdpfans.com
Thread-1 200 http://www.qq.com
Thread-5 200 http://www.newrank.cn
Thread-4 200 http://www.thinkphp.cn
Thread-3 200 http://www.mbalib.com
Thread-5 200 http://www.ceair.com
Thread-1 200 http://www.naver.com
Thread-4 200 http://www.2345.com
Thread-3 200 http://www.wanfangdata.com.cn
Thread-4 200 http://www.baoku.com
Thread-5 200 http://www.zimuku.net
Thread-1 200 http://www.taobao.com
Thread-4 200 http://www.tiancity.com
Thread-2 200 http://www.surveymonkey.com
Thread-3 200 http://www.ibuying.com
Thread-2 200 http://www.tower.im
Thread-2 Error: HTTPConnectionPool(host='www.baiducontent.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<u

Thread-5 200 http://www.lenovo.com
Thread-5 200 http://www.longau.com
Thread-5 200 http://www.seedit.com
Thread-4 200 http://www.5acbd.com
Thread-5 200 http://www.ofweek.com
Thread-5 200 http://www.61baobao.com
Thread-4 200 http://www.atobo.com.cn
Thread-5 200 http://www.400.cn
Thread-4 200 http://www.kubo365.com
Thread-5 200 http://www.wines-info.com
Thread-4 200 http://www.111cn.net
Thread-5 200 http://www.innisfree.com
Thread-5 200 http://www.weather.com.cn
Thread-2 200 http://www.joox.com
Thread-5 200 http://www.che168.com
Thread-5 Error: HTTPConnectionPool(host='www.dilidili.wang', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000243398F7400>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Thread-2 200 http://www.178.com
Thread-5 200 http://www.7po.com
Thread-4 200 http://www.zhongmin.cn
Thread-5 200 http://www.qiushibaike.com
Thread-5 200 http://www.9r.cn
Thread-2 200 http://w

Thread-2 200 http://www.51job.com
Thread-5 200 http://www.zhipin.com
Thread-1 200 http://www.youku.com
Thread-4 200 http://www.haosou.com
Thread-5 404 http://www.cgdc.com.cn
Thread-2 200 http://www.91jm.com
Thread-3 200 http://www.baixing.com
Thread-4 200 http://www.ishuhui.com
Thread-2 200 http://www.2cto.com
Thread-4 200 http://www.paopaoche.net
Thread-1 200 http://www.zhihu.com
Thread-5 200 http://www.61learn.com
Thread-1 200 http://www.cctv.com
Thread-4 200 http://www.csai.cn
Thread-1 200 http://www.airasia.com
Thread-4 200 http://www.zhaoshangbao.com
Thread-5 200 http://www.sm.cn
Thread-1 200 http://www.douyu.com
Thread-5 200 http://www.571xz.com
Thread-4 200 http://www.eol.cn
Thread-4 200 http://www.excelhome.net
Thread-5 Error: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
Thread-1 200 http://www.babytree.com
Thread-2 200 http://www.qoo10.com
Thread-4 200 http://www.missevan.com
Thread-3 200 http://www.258.com
Thread-4 200 http://ww

Thread-3 200 http://www.52pojie.cn
Thread-3 200 http://www.miwifi.com
Thread-4 200 http://www.b2b168.com
Thread-4 200 http://www.pingan.com
Thread-4 200 http://www.pushauction.com
Thread-3 200 http://www.iteye.com
Thread-4 200 http://www.sdo.com
Thread-4 200 http://www.9978.cn
Thread-3 200 http://www.kanzhun.com
Thread-4 200 http://www.ltaaa.com
Thread-3 200 http://www.mango.com
Thread-3 200 http://www.cheaa.com
Thread-4 200 http://www.gxyj.com
Thread-3 200 http://www.13322.com
Thread-4 200 http://www.kuaizhan.com
Thread-4 200 http://www.airchina.com.cn
Thread-3 200 http://www.jikexueyuan.com
Thread-3 Error: HTTPConnectionPool(host='www.taisha.org', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000243388CF278>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Thread-4 200 http://www.gcl-power.com
Thread-4 200 http://www.medsci.cn
Thread-3 200 http://www.mydigit.cn
Thread-3 200 http://

Thread-3 200 http://www.ca800.com
Thread-1 200 http://www.aliyun.com
Thread-3 200 http://www.dgtle.com
Thread-4 200 http://www.cicpa.org.cn
Thread-1 200 http://www.eastday.com
Thread-1 200 http://www.youdao.com
Thread-5 200 http://www.trendmicro.com
Thread-4 200 http://www.wowenda.com
Thread-1 200 http://www.oeeee.com
Thread-1 200 http://www.ci123.com
Thread-1 200 http://www.baike.com
Thread-3 200 http://www.cloudscar.com
Thread-1 200 http://www.adobe.com
Thread-1 200 http://www.rednet.cn
Thread-1 200 http://www.iqiyi.com
Thread-3 Error: HTTPSConnectionPool(host='www.bdmodel.cn', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.bdmodel.cn' doesn't match either of 'www.bdhome.cn', 'images.asky.cn', 'www.asky.cn', 'images.bdhome.cn', 'zsjs.searchs.cn', 'm.bdhome.cn', 'm.searchs.cn', 'www.searchs.cn', 'i.asky.cn', 'fangan.searchs.cn', 'm.asky.cn', 'bdhome.cn', 'i.bdhome.cn', 'images.searchs.cn', 'i.searchs.cn'",),))
Thread-3 200 http://www.ne

Thread-1 200 http://www.firefoxchina.cn
Thread-2 200 http://www.800hr.com
Thread-2 200 http://www.iconfont.cn
Thread-2 200 http://www.youzan.com
Thread-2 200 http://www.360kan.com
Thread-2 200 http://www.chinabyte.com
Thread-1 200 http://www.360doc.com
Thread-2 200 http://www.samsung.com
Thread-1 200 http://www.xunlei.com
Thread-2 200 http://www.zxart.cn
Thread-2 200 http://www.gucheng.com
Thread-2 200 http://www.bootcss.com
Thread-2 200 http://www.cankaoxiaoxi.com
Thread-2 200 http://www.58pic.com
Thread-2 200 http://www.81.cn
Thread-2 200 http://www.csair.com
Thread-5 Error: HTTPConnectionPool(host='www.xunyingwang.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000243388C0C88>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))
Thread-5 200 http://www.chinagate.cn
Thread-2 200 http://www.chiphell.com
Thread-2 200 http://www.antpedia.com
Thread-5 200 http://www

### 使用Queue的多线程爬虫
#### FIFO（先入后出）队列Queue、LIFO（后入先出）队列LifoQueue
#### 这1000个网页放入Queue的队列中，各个线程都是从这个队列中获取链接，知道完成所有的网页抓取为止。

In [5]:
link_list=[]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
with open('alexa.txt','r') as file:
    file_list=file.readlines()
    for eachone in file_list:
        link=eachone.split('\t')[1]
        link=link.replace('\n','')
        link_list.append(link)
    file.close()
    
start=time.time()
class myThread(threading.Thread):
    def __init__(self,name,q):
        threading.Thread.__init__(self)
        self.name=name
        self.q=q
    def run(self):
        print("Starting:"+self.name)
        while True:
            try:
                crawler(self.name,self.q)
            except:
                break
        print("Exiting:"+self.name)
def crawler(threadName,q):
    url=q.get(timeout=2) #取出队列中的url
    try:
        r=requests.get(url,headers=headers,timeout=20)
        print(q.qsize(),threadName,r.status_code,url)
    except Exception as e:
        print(q.qsize(),threadName,url,'Error:',e)

threadList=['Thread-1','Thread-2','Thread-3','Thread-4','Thread-5']
workQueue=queue.Queue(1000) #创建容纳1000个位置的队列
threads=[]

#创建新线程
for tName in threadList:
    thread=myThread(tName,workQueue)
    thread.start()
    threads.append(thread)

#填充队列
for url in link_list:
    workQueue.put(url) #将url传入队列中
    
#等待所有线程完成
for t in threads:
    t.join()
    
end=time.time()
print('Queue多线程爬虫的总时间为：',end-start)
print('Exiting Main Thread')

Starting:Thread-1Starting:Thread-2

Starting:Thread-3Starting:Thread-4

Starting:Thread-5
995 Thread-1 200 http://www.baidu.com
994 Thread-3 200 http://www.naver.com
993 Thread-2 200 http://www.qq.com
992 Thread-1 200 http://www.sohu.com
991 Thread-4 200 http://www.taobao.com
990 Thread-4 200 http://www.jd.com
989 Thread-3 200 http://www.tmall.com
988 Thread-2 200 http://www.sina.com.cn
987 Thread-4 200 http://www.360.cn
986 Thread-2 200 http://www.aliexpress.com
985 Thread-2 200 http://www.alipay.com
984 Thread-4 200 http://www.linkedin.com
983 Thread-2 200 http://www.hao123.com
982 Thread-2 200 http://www.youth.cn
981 Thread-4 200 http://www.csdn.net
980 Thread-4 200 http://www.tianya.cn
979 Thread-2 200 http://www.live.com
978 Thread-2 200 http://www.office.com
977 Thread-2 200 http://www.soso.com
976 Thread-2 200 http://www.so.com
975 Thread-2 200 http://www.gmw.cn
974 Thread-3 200 http://www.weibo.com
973 Thread-2 200 http://www.china.com
972 Thread-2 200 http://www.huaban.com
971

826 Thread-2 200 http://www.umeng.com
825 Thread-2 200 http://www.hupu.com
824 Thread-4 200 http://www.qcloud.com
823 Thread-4 200 http://www.3dmgame.com
822 Thread-4 200 http://www.icolor.com.cn
821 Thread-4 200 http://www.360.com
820 Thread-5 200 http://www.klm.com
819 Thread-5 200 http://www.miui.com
818 Thread-4 200 http://www.36kr.com
817 Thread-5 200 http://www.boc.cn
816 Thread-5 200 http://www.joyme.com
815 Thread-4 200 http://www.gamersky.com
814 Thread-2 403 http://www.ikanman.com
813 Thread-5 200 http://www.17173.com
812 Thread-4 200 http://www.uc.cn
811 Thread-2 200 http://www.alimama.com
810 Thread-2 200 http://www.cnr.cn
809 Thread-2 200 http://www.miomio.tv
808 Thread-4 200 http://www.focus.cn
807 Thread-2 200 http://www.jjwxc.net
806 Thread-2 200 http://www.hjenglish.com
805 Thread-2 200 http://www.dangdang.com
804 Thread-4 http://www.5dcar.com Error: HTTPConnectionPool(host='www.5dcar.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urll

647 Thread-4 200 http://www.zybang.com
646 Thread-1 200 http://www.ooopic.com
645 Thread-4 200 http://www.gw-ec.com
644 Thread-3 200 http://www.fang.com
643 Thread-3 200 http://www.ithome.com
642 Thread-1 200 http://www.wed114.cn
641 Thread-4 200 http://www.huomao.com
640 Thread-2 200 http://www.free.com.tw
639 Thread-1 200 http://www.chinanews.com
638 Thread-4 200 http://www.doc88.com
637 Thread-4 200 http://www.icbc.com.cn
636 Thread-1 200 http://www.evaair.com
635 Thread-2 200 http://www.sanguosha.com
634 Thread-4 200 http://www.youxidudu.com
633 Thread-1 200 http://www.verycd.com
632 Thread-2 200 http://www.netcoc.com
631 Thread-2 200 http://www.liaoxuefeng.com
630 Thread-1 200 http://www.dygang.com
629 Thread-2 200 http://www.flyasiana.com
628 Thread-1 200 http://www.sciencenet.cn
627 Thread-1 200 http://www.800hr.com
626 Thread-2 200 http://www.feiyang.com
625 Thread-1 200 http://www.iconfont.cn
624 Thread-2 200 http://www.youzan.com
623 Thread-1 200 http://www.360kan.com
622 Thr

494 Thread-2 200 http://www.home77.com
493 Thread-2 200 http://www.xdowns.com
492 Thread-1 200 http://www.gao7.com
491 Thread-5 200 http://www.qunar.com
490 Thread-5 http://www.szjy188.com Error: HTTPSConnectionPool(host='www.szjy188.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))
489 Thread-3 http://www.ccb.com.cn Error: HTTPConnectionPool(host='www.ccb.com.cn', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000024339B43080>, 'Connection to www.ccb.com.cn timed out. (connect timeout=20)'))
488 Thread-3 200 http://www.135editor.com
487 Thread-1 200 http://www.sgcn.com
486 Thread-5 200 http://www.tuniu.com
485 Thread-3 http://www.f.com Error: HTTPConnectionPool(host='www.f.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.H

349 Thread-1 200349 http://www.dhl.com
 Thread-3 200 http://www.51nb.com
347 Thread-5 200 http://www.xici.net
346 Thread-1 200 http://www.cnki.com.cn
345 Thread-3 200 http://www.redocn.com
344 Thread-2 200 http://www.122.gov.cn
343 Thread-5 403 http://www.qvc.com
342 Thread-1 200 http://www.aipai.com
341 Thread-3 200 http://www.dapenti.com
340 Thread-3 200 http://www.tgfcer.com
339 Thread-1 200 http://www.jiankang.com
338 Thread-5 200 http://www.guidechem.com
337 Thread-1 200 http://www.sodao.com
336 Thread-5 200 http://www.zhcw.com
335 Thread-5 200 http://www.ablesky.com
334 Thread-2 200 http://www.3lian.com
333 Thread-2 200 http://www.7k7k.com
332 Thread-3 200 http://www.freebuf.com
331 Thread-5 200 http://www.microsoftstore.com.cn
330 Thread-2 200 http://www.southmoney.com
329 Thread-2 200 http://www.meitu.com
328 Thread-5 200 http://www.digitaling.com
327 Thread-2 200 http://www.chinaaet.com
326 Thread-5 200 http://www.kaoyan.com
325 Thread-2 200 http://www.aipai.com
324 Thread-1 2

216 Thread-3 200 http://www.313.cn
215 Thread-3 http://www.ssl-images-amazon.com Error: HTTPConnectionPool(host='www.ssl-images-amazon.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024339AF67F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
214 Thread-3 200 http://www.eepw.com.cn
213 Thread-3 http://www.gs307.com Error: HTTPConnectionPool(host='www.gs307.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024339B6CB70>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
212 Thread-3 http://www.yindou.com Error: HTTPConnectionPool(host='www.yindou.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024339AF6CC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
211 Thre

94 Thread-5 200 http://www.kekenet.com
93 Thread-1 200 http://www.21csp.com.cn
92 Thread-4 200 http://www.c5game.com
91 Thread-1 200 http://www.baofeng.com
90 Thread-5 200 http://www.juejin.im
89 Thread-4 200 http://www.kuwo.cn
88 Thread-1 200 http://www.6.cn
87 Thread-4 200 http://www.sanwen.net
86 Thread-1 200 http://www.962.net
85 Thread-5 200 http://www.chayu.com
84 Thread-1 200 http://www.innisfree.com
83 Thread-4 200 http://www.etest.net.cn
82 Thread-4 200 http://www.lawtime.cn
81 Thread-5 200 http://www.dragonair.com
80 Thread-4 200 http://www.sccnn.com
79 Thread-1 200 http://www.vjshi.com
78 Thread-5 200 http://www.qqbaobao.com
77 Thread-4 200 http://www.dragonair.com
76 Thread-5 200 http://www.lawtime.cn
75 Thread-1 200 http://www.vjshi.com
74 Thread-4 200 http://www.sccnn.com
73 Thread-5 200 http://www.qqbaobao.com
72 Thread-5 200 http://www.cntv.cn
71 Thread-5 200 http://www.knowsky.com
70 Thread-4 200 http://www.5118.com
69 Thread-4 200 http://www.wrz.com
68 Thread-5 200 ht

In [23]:
link_list=[]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
with open('alexa.txt','r') as file:
    file_list=file.readlines()
    for eachone in file_list:
        link=eachone.split('\t')[1]
        link=link.replace('\n','')
        link_list.append(link)
    file.close()

start_time=time.time()
for i,url in enumerate(link_list):
    try:
        r=requests.get(url,headers=headers)
        print('爬到第{}个url, 状态码为:{},url为:{}'.format(i+1,r.status_code,url))
    except Exception as e:
        print(url,e)
end_time=time.time()
print(end_time-start_time)

爬到第1个url, 状态码为:200,url为:http://www.baidu.com
爬到第2个url, 状态码为:200,url为:http://www.qq.com
爬到第3个url, 状态码为:200,url为:http://www.naver.com
爬到第4个url, 状态码为:200,url为:http://www.taobao.com
http://www.reddit.com HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),))
爬到第6个url, 状态码为:200,url为:http://www.sohu.com
爬到第7个url, 状态码为:200,url为:http://www.tmall.com
爬到第8个url, 状态码为:200,url为:http://www.sina.com.cn
http://www.daum.net HTTPConnectionPool(host='www.daum.net', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015D3FDBC240>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))
爬到第10个url, 状态码为:200,url为:http://www.jd.com
爬到第11个url, 状态码为:200,url为:http://www.360.cn
爬到第12个url, 状态码为:200,url为:http://www.weibo.com
爬到第13个url, 状态码为:200,url为:http://www.aliexpress.com
爬到第14个url, 

爬到第148个url, 状态码为:200,url为:http://www.hotelscombined.com
爬到第149个url, 状态码为:200,url为:http://www.mydrivers.com
爬到第150个url, 状态码为:200,url为:http://www.taoche.com
爬到第151个url, 状态码为:200,url为:http://www.runoob.com
爬到第152个url, 状态码为:200,url为:http://www.tlscontact.com
爬到第153个url, 状态码为:200,url为:http://www.nba.com
http://www.gamebase.com.tw ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
爬到第155个url, 状态码为:200,url为:http://www.zhibo8.cc
爬到第156个url, 状态码为:200,url为:http://www.hexun.com
爬到第157个url, 状态码为:200,url为:http://www.xiami.com
爬到第158个url, 状态码为:200,url为:http://www.finnair.com
爬到第159个url, 状态码为:200,url为:http://www.feng.com
爬到第160个url, 状态码为:200,url为:http://www.cdstm.cn
爬到第161个url, 状态码为:200,url为:http://www.uniqlo.com
爬到第162个url, 状态码为:200,url为:http://www.iciba.com
爬到第163个url, 状态码为:200,url为:http://www.qudong.com
爬到第164个url, 状态码为:200,url为:http://www.panda.tv
爬到第165个url, 状态码为:200,url为:http://www.cnbeta.com
爬到第166个url, 状态码为:200,url为:http://www.nipic.com
爬到第167个url, 状态

爬到第303个url, 状态码为:200,url为:http://www.shop.com
爬到第304个url, 状态码为:200,url为:http://www.588ku.com
爬到第305个url, 状态码为:200,url为:http://www.le.com
爬到第306个url, 状态码为:200,url为:http://www.sina.com
爬到第307个url, 状态码为:200,url为:http://www.jstv.com
爬到第308个url, 状态码为:200,url为:http://www.ceconline.com
爬到第309个url, 状态码为:200,url为:http://www.koreanair.com
爬到第310个url, 状态码为:200,url为:http://www.skype.com
爬到第311个url, 状态码为:200,url为:http://www.ih5.cn
爬到第312个url, 状态码为:200,url为:http://www.ems.com.cn
爬到第313个url, 状态码为:200,url为:http://www.efu.com.cn
爬到第314个url, 状态码为:200,url为:http://www.pcbaby.com.cn
爬到第315个url, 状态码为:200,url为:http://www.shimo.im
爬到第316个url, 状态码为:200,url为:http://www.macaolife.com
爬到第317个url, 状态码为:200,url为:http://www.xiu.com
爬到第318个url, 状态码为:200,url为:http://www.eastmoney.com
爬到第319个url, 状态码为:200,url为:http://www.xiumi.us
爬到第320个url, 状态码为:200,url为:http://www.yhd.com
爬到第321个url, 状态码为:200,url为:http://www.jiemian.com
爬到第322个url, 状态码为:200,url为:http://www.daikuan.com
爬到第323个url, 状态码为:200,url为:http://www.ximalaya.com

爬到第447个url, 状态码为:200,url为:http://www.pearsoncmg.com
爬到第448个url, 状态码为:200,url为:http://www.kongfz.com
爬到第449个url, 状态码为:200,url为:http://www.365jilin.com
爬到第450个url, 状态码为:200,url为:http://www.strawberrynet.com
爬到第451个url, 状态码为:200,url为:http://www.11467.com
爬到第452个url, 状态码为:200,url为:http://www.jobui.com
爬到第453个url, 状态码为:200,url为:http://www.hh010.com
爬到第454个url, 状态码为:200,url为:http://www.teambition.com
爬到第455个url, 状态码为:200,url为:http://www.woshipm.com
爬到第456个url, 状态码为:200,url为:http://www.lge.com
http://www.kanxi.cc HTTPConnectionPool(host='www.kanxi.cc', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015D3FFDFA90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
爬到第458个url, 状态码为:200,url为:http://www.leiphone.com
爬到第459个url, 状态码为:403,url为:http://www.d1com.com
http://www.114so.cn HTTPConnectionPool(host='www.114so.cn', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<u

爬到第572个url, 状态码为:200,url为:http://www.33lc.com
爬到第573个url, 状态码为:200,url为:http://www.thepaper.cn
爬到第574个url, 状态码为:200,url为:http://www.scswl.cn
爬到第575个url, 状态码为:200,url为:http://www.officedepot.com
爬到第576个url, 状态码为:200,url为:http://www.fx678.com
爬到第577个url, 状态码为:200,url为:http://www.banma.com
爬到第578个url, 状态码为:200,url为:http://www.eee114.com
爬到第579个url, 状态码为:200,url为:http://www.9384.com
爬到第580个url, 状态码为:200,url为:http://www.xuexila.com
爬到第581个url, 状态码为:200,url为:http://www.9384.com
爬到第582个url, 状态码为:200,url为:http://www.xuexila.com
爬到第583个url, 状态码为:200,url为:http://www.cheshen.cn
爬到第584个url, 状态码为:200,url为:http://www.mr-world.com
爬到第585个url, 状态码为:200,url为:http://www.fx112.com
爬到第586个url, 状态码为:200,url为:http://www.97665.com
爬到第587个url, 状态码为:200,url为:http://www.chinahr.com
爬到第588个url, 状态码为:200,url为:http://www.acs.org
爬到第589个url, 状态码为:200,url为:http://www.mikecrm.com
http://www.checheng.com HTTPConnectionPool(host='www.checheng.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionErro

爬到第699个url, 状态码为:200,url为:http://www.zhaoshangbao.com
爬到第700个url, 状态码为:200,url为:http://www.eol.cn
爬到第701个url, 状态码为:200,url为:http://www.excelhome.net
爬到第702个url, 状态码为:200,url为:http://www.missevan.com
爬到第703个url, 状态码为:200,url为:http://www.cncv.org.cn
爬到第704个url, 状态码为:200,url为:http://www.365yg.com
爬到第705个url, 状态码为:200,url为:http://www.huim.com
爬到第706个url, 状态码为:200,url为:http://www.zxxk.com
http://www.51yes.com HTTPConnectionPool(host='www.51yes.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015D3FFDFDA0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))
爬到第708个url, 状态码为:200,url为:http://www.cainiao.com
http://www.nh87.cn ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
http://www.b0yp.com HTTPConnectionPool(host='www.b0yp.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.H

爬到第802个url, 状态码为:200,url为:http://www.ef43.com.cn
爬到第803个url, 状态码为:200,url为:http://www.newrank.cn
爬到第804个url, 状态码为:200,url为:http://www.ceair.com
爬到第805个url, 状态码为:200,url为:http://www.zimuku.net
爬到第806个url, 状态码为:200,url为:http://www.ppkoo.com
爬到第807个url, 状态码为:200,url为:http://www.jc35.com
爬到第808个url, 状态码为:200,url为:http://www.dnspod.cn
爬到第809个url, 状态码为:200,url为:http://www.hsw.cn
爬到第810个url, 状态码为:200,url为:http://www.caixin.com
爬到第811个url, 状态码为:200,url为:http://www.manmanbuy.com
http://www.23us.com HTTPConnectionPool(host='www.23us.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015D3FF69F98>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
爬到第813个url, 状态码为:200,url为:http://www.asus.com
爬到第814个url, 状态码为:200,url为:http://www.zoosnet.net
爬到第815个url, 状态码为:200,url为:http://www.xp510.com
爬到第816个url, 状态码为:200,url为:http://www.vgtime.com
爬到第817个url, 状态码为:200,url为:http://www.qiushibaike.com
爬到第818个

爬到第932个url, 状态码为:200,url为:http://www.wasu.cn
爬到第933个url, 状态码为:200,url为:http://www.mojifen.com
爬到第934个url, 状态码为:200,url为:http://www.nvidia.com
爬到第935个url, 状态码为:200,url为:http://www.oceanpark.com.hk
爬到第936个url, 状态码为:200,url为:http://www.pcbeta.com
爬到第937个url, 状态码为:200,url为:http://www.psnine.com
爬到第938个url, 状态码为:200,url为:http://www.228.com.cn
爬到第939个url, 状态码为:200,url为:http://www.zhuixinfan.com
爬到第940个url, 状态码为:200,url为:http://www.okcoin.cn
爬到第941个url, 状态码为:200,url为:http://www.huya.com
爬到第942个url, 状态码为:200,url为:http://www.1ppt.com
爬到第943个url, 状态码为:200,url为:http://www.fyber.com
爬到第944个url, 状态码为:200,url为:http://www.72byte.com
爬到第945个url, 状态码为:200,url为:http://www.cpic.com.cn
爬到第946个url, 状态码为:200,url为:http://www.wlmq.com
爬到第947个url, 状态码为:200,url为:http://www.lusongsong.com
爬到第948个url, 状态码为:200,url为:http://www.fanjian.net
爬到第949个url, 状态码为:200,url为:http://www.hopetrip.com.hk
爬到第950个url, 状态码为:200,url为:http://www.hnjy.com.cn
爬到第951个url, 状态码为:200,url为:http://www.8kana.com
爬到第952个url, 状态码为:200,url为:htt

### 多进程爬虫
#### 1，使用multiprocessing的多进程爬虫
#### 2，使用Pool+Queue的多进程爬虫  (进程池)

In [3]:
#查看cpu个数
print(cpu_count())

4


In [23]:
link_list=[]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
with open("alexa.txt",'r') as file:
    file_list=file.readlines()
    for eachone in file_list:
        link=eachone.split('\t')[1]
        link=link.replace('\n','')
        link_list.append(link)
    file.close()
    
start=time.time()
class myProcess(Process):
    def __init__(self,q):
        Process.__init__(self)
        self.q=q
    def run(self):
        print('Starting',self.pid)
        while not self.q.empty():
            crawler(self.q)
        print('Exiting',self.pid)
    

def crawler(q):
    url=q.get(timeout=2)
    try:
        r=requests.get(url,headers=headers,timeout=20)
        print(q.qsize(),r.status_code,url)
    except Exception as e:
        print(q.qsize(),url,'Error:',e)
        
if __name__ == '__main__':
    ProcessName=['Process-1','Process-2','Process-3']
    workQueue=Queue(1000)
    ProcessList=[]
    #填充队列
    for url in link_list:
        workQueue.put(url)

    for i in range(0,3):
        p=myProcess(workQueue)
        p.daemon=True #当父进程结束后，子进程就会自动停止
        p.start()
        ProcessList.append(p)
    for p in ProcessList:
        p.join()
    end=time.time()
    print('Process+Queue多进程爬虫的总时间为：',end-start)
    print('Main Process Ended')

Starting None
999 200 http://www.baidu.com
998 200 http://www.qq.com
997 200 http://www.naver.com
996 200 http://www.taobao.com
995 http://www.reddit.com Error: HTTPConnectionPool(host='www.reddit.com', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000256C69092B0>, 'Connection to www.reddit.com timed out. (connect timeout=20)'))
994 200 http://www.sohu.com
993 200 http://www.tmall.com
992 200 http://www.sina.com.cn
991 http://www.daum.net Error: HTTPConnectionPool(host='www.daum.net', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000256C67D5860>, 'Connection to www.daum.net timed out. (connect timeout=20)'))
990 200 http://www.jd.com
989 200 http://www.360.cn
988 200 http://www.weibo.com
987 200 http://www.aliexpress.com
986 200 http://www.linkedin.com
985 200 http://www.alipay.com
984 200 http://www.hao123.com
983 200 http://www.csdn

KeyboardInterrupt: 

In [2]:
#使用Pool+Queue的多进程爬虫
# 非阻塞：不一定非要等待结果出来之后才添加其他进程运行
# 阻塞：要等到回调结果出来，在有结果出来之前，当前进程会被挂起

link_list=[]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
with open("alexa.txt",'r') as file:
    file_list=file.readlines()
    for eachone in file_list:
        link=eachone.split('\t')[1]
        link=link.replace('\n','')
        link_list.append(link)
    file.close()
    
start=time.time()
def crawler(q,index):
    Process_id='Process-'+str(index)
    while not q.empty():
        url=q.get(timeout=2)
        try:
            r=requests.get(url,headers=headers,timeout=20)
            pritn(Process_id,q.qsize(),r.status_code,url)
        except Exception as e:
            print(Process_id,q.qsize(),url,'Error:',e)


manger=Manager()
workQueue=manger.Queue(1000)
    
    #填充队列
for url in link_list:
    workQueue.put(url)
        
pool=Pool(processes=3)
for i in range(4):
    pool.apply_async(crawler,args=(workQueue,i))

print('started processes')
pool.close()
pool.join()
    
end=time.time()
print("Pool+Queue多进程爬虫的总时间为：",end-start)
print('打完收工 ')

started processes


AssertionError: 

In [13]:
url = 'https://bbs.hupu.com/bxj'
r = requests.get(url,headers = headers)
soup = BeautifulSoup(r.text)

In [18]:
lis = soup.find('ul',class_="for-list").find_all('li')

In [23]:
li = lis[0]
li.div.a.get('href'),li.div.a.text

('/28222550.html', '虎扑App视频列表改动满意度打分及建议征求')

In [27]:
soup.select('#ajaxtable > div.show-list > ul > li > div.titlelink.box > a')

[<a class="truetit" href="/28222550.html" target="_blank">虎扑App视频列表改动满意度打分及建议征求</a>,
 <a class="truetit" href="/28240026.html" target="_blank">试试吴尊在深圳开的汉堡店</a>,
 <a class="truetit" href="/28237187.html" target="_blank">离婚了在一起吃饭正常么。</a>,
 <a class="truetit" href="/28224239.html" target="_blank">钱 真的是一个很现实的问题</a>,
 <a class="truetit" href="/28240613.html" target="_blank">逛咸鱼看到这个“猪宝”要卖800万，什么水平？</a>,
 <a class="truetit" href="/28239887.html" target="_blank">老婆胀奶有什么好办法吗</a>,
 <a class="truetit" href="/28238110.html" target="_blank">你拿一张动物萌照来能打败它吗？ zt</a>,
 <a class="truetit" href="/28236134.html" target="_blank">大神教你玩一手好牌，学会了聚会时给朋友们露一手！ zt</a>,
 <a class="truetit" href="/28240842.html" target="_blank">女朋友总是提起我前女友是啥心态？</a>,
 <a class="truetit" href="/28236407.html" target="_blank">史上最嚣张!他站原地5分钟抢劫两次 zt</a>,
 <a class="truetit" href="/28233585.html" target="_blank">白色轿车是全责还是无责，黑车应该是无责吧？ zt</a>,
 <a class="truetit" href="/28240600.html" target="_blank">帮女神表姐找对象，优秀的JRS请进</a>,
 <a class="truetit

In [28]:
htlm = etree.HTML(r.text)

In [34]:
htlm.xpath('//*[@id="ajaxtable"]/div[1]/ul/li[2]/div[1]/a/text()')

['试试吴尊在深圳开的汉堡店']

### 修改爬虫的时间

In [36]:
print(time.time())
sleep_time=random.randint(0,2)+random.random()
print(sleep_time)
time.sleep(sleep_time)
print(time.time())

1562057371.4412844
2.7509459483822596
1562057374.194027


In [22]:
url="http://w3school.com.cn/"
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
r=requests.get(url)
soup=BeautifulSoup(r.content.decode('gbk'),'lxml')
x=soup.find('p',id='p1').text
print(x)


W3School 简体中文版提供的内容仅用于培训和测试，不保证内容的正确性。通过使用本站内容随之而来的风险与本站无关。



#### 看 charset=什么就是什么编码
#### 当charset=‘utf-8’时还是出现乱码，则可能是网页是使用gzip将网页压缩了，
#### 使用r.content会自动编码gzip和deflate传输编码的响应数据

In [75]:
url='http://www.sina.com.cn'
r= requests.get(url,headers=headers)
# print(r.text)
after_gzip=r.content.decode('utf-8')
print(after_gzip)

<!DOCTYPE html>
<!-- [ published at 2019-07-02 18:30:02 ] -->
<html>
<head>
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>新浪首页</title>
	<meta name="keywords" content="新浪,新浪网,SINA,sina,sina.com.cn,新浪首页,门户,资讯" />
	<meta name="description" content="新浪网为全球用户24小时提供全面及时的中文资讯，内容覆盖国内外突发新闻事件、体坛赛事、娱乐时尚、产业资讯、实用信息等，设有新闻、体育、娱乐、财经、科技、房产、汽车等30多个内容频道，同时开设博客、视频、论坛等自由互动交流空间。" />
	<meta content="always" name="referrer">
    <link rel="mask-icon" sizes="any" href="//www.sina.com.cn/favicon.svg" color="red">
	<meta name="stencil" content="PGLS000022" />
	<meta name="publishid" content="30,131,1" />
	<meta name="verify-v1" content="6HtwmypggdgP1NLw7NOuQBI2TW8+CfkYCoyeB8IDbn8=" />
	<meta name="360-site-verification" content="63349a2167ca11f4b9bd9a8d48354541" />
	<meta name="application-name" content="新浪首页"/>
	<meta name ="msapplication-TileImage" content="//i1.sinaimg.cn/dy/deco/2013/0312/logo.png"/>
	<meta nam

### 读写文件的中文乱码

In [80]:
title='我love你'
# with open('title.json','w',encoding='utf-8',) as f:
#     json.dump([title],f)
    
#显示中文
with open('title.json','w',encoding='utf-8',) as f:
    json.dump([title],f,ensure_ascii=False)  

## 登录
### 1,处理登录表单
### 2,处理cookies，让网页记住你的登录
#### cookies大概会定义4个参数：
##### Set-Cookie:name = VALUE;     cookie的名称（一般都进行加密处理了）
##### expires = DATE;        (是cookie的到期日期)
##### path = PATH;         (是cookie的路径)
##### domain = DOMAIN_NAME;     (是cookie所在的域名)

In [96]:
session=requests.session()

post_url='http://www.santostang.com/wp-login.php'
post_data={'log':'test',
          'pwd':'a12345',
          'rememberme':'forever',
          "redirect_to":"http://www.santostang.com/wp-admin/",
          "testcookie":1}
login=session.post(post_url,post_data,headers=headers)
print(login.status_code)

200


In [99]:
session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')

post_url='http://www.santostang.com/wp-login.php'
post_data={'log':'test',
          'pwd':'a12345',
          'rememberme':'forever',
          "redirect_to":"http://www.santostang.com/wp-admin/",
          "testcookie":1}

login_page = session.post(post_url, data=postdata, headers=headers)
print(login_page.status_code)
session.cookies.save()

200


### 完整的登录以及免密登录代码

In [102]:
session=requests.session()
session.cookies=cookielib.LWPCookieJar(filename='cookies')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('0')
    
def isLogin():
    #通过查看用户个人信息来判断是否已经登录
    url='http://www.santostang.com/wp-admin/profile.php'
    login_code=session.get(url,headers=headers,allow_redirects=False).status_code
    if login_code == 200:
        return True
    else:
        return False
    
def login(account,secret):
    post_url='http://www.santostang.com/wp-login.php'
    post_data={'log':'account',
          'pwd':'secret',
          'rememberme':'forever',
          "redirect_to":"http://www.santostang.com/wp-admin/",
          "testcookie":1}
    try:
        #不需要验证码直接登录
        login_page=session.post(post_url,post_data,headers=headers)
        login_code=login_page.text
        print(login_page.status_code)
    except:
        print('0')
    session.cookies.save()
    
    
if __name__ == '__main__':
    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
    if isLogin():
        print('你已经登录了')
    else:
        print('呵呵')
        #login('test','a12345')

你已经登录了


## 知乎live

In [6]:
link='https://api.zhihu.com/lives/homefeed?includes=live'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(link,headers=headers)
j=json.loads(r.text)
next_page=j['paging']['next']
is_end=j['paging']['is_end']
print(next_page)
print(is_end)

https://api.zhihu.com/lives/homefeed?limit=1000&offset=1000
True


In [4]:
url='https://api.zhihu.com/lives/homefeed?limit=20000&offset=0'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
r=requests.get(url,headers=headers)
j=json.loads(r.text)
subject_list=[]
for i in range(len(j['data'])):
    subject=j['data'][i]['live']['subject']
    subject_list.append(subject)

subject_list

['得了脚气和灰指甲应该怎么办？',
 '自学配音赚外快？我教你！',
 '雅思口语满分经验谈',
 '视网膜脱落，离年轻的你有多远？',
 '北大才子：如何一年读  500 本书',
 '让姑娘们受益一生的重要妇科常识',
 '省前20教你如何高考政治近满分',
 '应届生如何拿到阿里前端 Offer',
 '带你逐层破解“多囊卵巢综合征”',
 '游泳中的水性和安全练习',
 '教你认识机械工程师的工作',
 '完美法律文书应该这样写',
 '咨询与金融，我该如何选择？',
 '名校计算机专业保研：谋定而后动',
 '语音识别技术的前世今生',
 '经济学趣史：古典经济学大咖',
 '美食摄影，是件严肃的事情',
 'ACCA考生的大学四年该怎么过？',
 '高考失利，复读时该如何调整心态、提高成绩？',
 '家庭保险配置与常见问题',
 '去堪察加半岛，爬火山钓鲑鱼',
 '习惯与计划，高效人士的必修课',
 '博物君教你成为养花高手',
 '打破会议中“没想法”的僵局',
 '你本不该近视',
 '游戏设计：易学性',
 '2019 房地产投资寒冬，还要不要入行？',
 '绘画零基础，我该从哪儿入手学素描？',
 '如何挖掘出有深度的好故事？',
 '财务分析的核心策略',
 '如何通过唱英文歌学英语发音',
 '学好英语半年足矣，只要做这 3 件小事',
 '萌新商业摄影师如何装dalao？',
 '打造精干职场形象，你只需要10分钟',
 '英语标准发音基础及进阶课程',
 '如何处理恋爱经费，打理好俩人的小金库？',
 '车尔尼599应该怎样学(2)',
 '如何购买人生中的第一支基金？',
 '【公益 Live】看清「疯狂的爱」，走出家暴阴影',
 '前端工程师如何不断自我提升',
 '新传考研：复旦学姐高分备考经验',
 '赴美行医：如何规划时间',
 '模型海军史讲座',
 '个人任务管理入门：基本概念与基础工具',
 'CPA长投如何一小时入门',
 '这样修改，你的文章脱胎换骨',
 '跟我淘遍全球最适合自己的香水',
 '朋友（自己）得了抑郁症怎么办？',
 '如何考入浙大热门专业研究生',
 '规范的跑后拉伸应该怎么做',
 '在外资投行巧用 Excel 做分析',
 '如何矫正不良步态？',
 '如何正确地背单词',
 '【公益 Liv